### Chains with multiple inputs

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain

llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

prompt_template = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input="a parrot")

/var/folders/x8/ps_xddr95734lgrb6gqpr7g80000gp/T/ipykernel_16368/1191947449.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


{'input': 'a parrot',
 'text': 'Why did the parrot wear a raincoat? \n\nBecause it wanted to be a poly-unsaturated!'}

In [3]:
prompt_template = PromptTemplate(input_variables=["input", "language"], template="Tell me a joke about {input} in {language}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke({"input": "a parrot", "language": "hindi"})

{'input': 'a parrot',
 'language': 'hindi',
 'text': 'बिलकुल! यहाँ एक तोता पर मजेदार चुटकुला है:\n\nएक तोता अपनी मम्मी से बोला, "मम्मी, मुझे सब्ज़ी नहीं चाहिए!"\nतो उसकी मम्मी ने कहा, "क्यों?"\nतोता बोला, "क्योंकि मैं हरा-भरा हूँ!" 😊'}

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [4]:
from langchain.chains.sequential import SequentialChain

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You ordered {dish_name} from {restaurantName} and your experience was {experience}. Write a review: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

#=============================================================================================================

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

#==============================================================================================================

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {comment}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

#================================================================================================================

# This is an LLMChain to translate a summary into German.
prompt_translation = PromptTemplate.from_template(
    template="""Translate the summary in 3 languages 
    
    1. Hindi
    2. French
    3. Spanish 
    
    \n\n 
    
    {summary}
    """
)
chain_translation = LLMChain(
    llm=llm, prompt=prompt_translation, output_key="hindi_translation"
)

#==================================================================================================================

In [5]:
overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name","restaurantName","experience"],
    output_variables=["review", "comment", "summary", "hindi_translation"],
)

overall_chain.invoke({"dish_name": "Chicken Biryani", "restaurantName":"Akbar Biryani", "experience": "It was awful!"})

{'dish_name': 'Chicken Biryani',
 'restaurantName': 'Akbar Biryani',
 'experience': 'It was awful!',
 'review': "**Review for Akbar Biryani:**\n\n⭐☆☆☆☆  (1/5)\n\nI recently ordered Chicken Biryani from Akbar Biryani, and I must say, my experience was nothing short of disappointing. As a biryani enthusiast, I was excited to try what I hoped would be a delightful dish, but unfortunately, it fell far short of expectations.\n\nFirstly, the flavor profile was lacking. A good biryani should have a balanced blend of spices that tantalize the palate, but what I received was bland and uninspiring. The chicken was overcooked and dry, which made it difficult to enjoy. Instead of succulent pieces infused with rich flavors, I was met with an unappetizing texture.\n\nAdditionally, the rice was undercooked and clumped together, making it a challenge to serve. There was little to no aroma that one typically associates with freshly made biryani. Presentation-wise, it didn’t look appealing either, which

Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

In [6]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

positive_template = """You are an AI that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them. \
Here is the text:
{input}"""

neutral_template = """You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}"""

In [7]:
prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]

In [8]:

# prompt_infos is expected to be a list of dictionaries, each containing at least "name" and "prompt_template" keys.
# For each prompt info:
#   Extract the name and prompt template.
#   Create a PromptTemplate object that takes one input variable called "input".
#   Create an LLMChain: This is an LLM + a prompt. So, each "destination" (e.g., "food_review", "hotel_review", etc.) gets its own LLMChain.
#   Store each chain in the destination_chains dictionary, keyed by its name.

# Result: A dictionary where each key is a destination name and each value is a ready-to-use LLMChain for that intent.
    

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
destination_chains

{'positive': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an AI that focuses on the positive side of things. Whenever you analyze a text, you look for the positive aspects and highlight them. Here is the text:\n{input}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10e5aa790>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10e59a710>, root_client=<openai.OpenAI object at 0x10e59bfd0>, root_async_client=<openai.AsyncOpenAI object at 0x10e5aa810>, model_name='gpt-4o-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}),
 'neutral': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an AI that has a neutral perspective. You just provide a balanced analysis of the te

In [9]:
# For each prompt in prompt_infos, create a string like "food_review: Handles food related reviews".
# Combine all such strings into a newline-separated list (destinations_str).
# Print it for debugging. This will help the router know what all "destinations" it can send queries to.

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

positive: Good for analyzing positive sentiments
neutral: Good for analyzing neutral sentiments
negative: Good for analyzing negative sentiments


In [12]:
# Use a predefined template (MULTI_PROMPT_ROUTER_TEMPLATE) that expects the available destinations.
# Plug in the list of destinations you just built.
# Build a PromptTemplate for the router, which will use the user input and output a routing decision.
# Set the output_parser to RouterOutputParser() (this parses the router’s decision).

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)


# Create a Router Chain that will use your LLM and the router prompt.
# When given an input, it decides which destination chain (from step 1) to route the input to.
router_chain = LLMRouterChain.from_llm(llm, router_prompt)


# MultiPromptChain is the master orchestrator.
# It uses:
# The router chain to select a destination.
# The dictionary of destination chains.
# A default chain (for "neutral" or unknown inputs).
# verbose=True gives you logs as it routes and processes.

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True,
)

chain.invoke({"input": "I ordered Pizza Salami for 9.99$ and it was not fresh!"})



> Entering new MultiPromptChain chain...
negative: {'input': 'I ordered Pizza Salami for $9.99 and it was not fresh!'}
> Finished chain.


{'input': 'I ordered Pizza Salami for $9.99 and it was not fresh!',
 'text': 'Here are the potential negative aspects found in the text:\n\n1. **Quality of Food**: The pizza ordered was described as "not fresh," which indicates a lack of quality in the food provided. This could lead to dissatisfaction and a negative dining experience.\n\n2. **Value for Money**: At $9.99, customers expect a certain level of quality and freshness. The perception that the pizza was not fresh may suggest that the price does not reflect the quality received.\n\n3. **Possible Health Concerns**: Consuming food that is not fresh could raise concerns about food safety and potential health issues.\n\n4. **Expectations vs. Reality**: The expectation of receiving a freshly made pizza was not met, leading to disappointment.\n\n5. **Customer Experience**: The overall experience of ordering and consuming the pizza was likely negative, which may impact future purchasing decisions from the same establishment.\n\n6. **P

In [13]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

tmpl = """You are a sentiment classifier. Determine if the input is Positive, Negative, or Neutral. 
Here is the text:
{input}"""

prompt_template = PromptTemplate(input_variables=["input"], template=tmpl)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke({"input": "I had a good time"})

APIConnectionError: Connection error.